In [1]:
import os
import lusid
import datetime
import pandas as pd
import lusid.models as models
from lusidtools.cocoon import load_from_data_frame
from lusidtools.cocoon.utilities import create_scope_id
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response
)
from lusidjam import RefreshingToken

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

print('LUSID Environment Initialised')
print('LUSID version : ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

# Define our scope and portfolio code
scope = "As_at_DEMO-"+ create_scope_id()
code = "PORT_0001"
print(f"Scope: {scope}")
print(f"Code: {code}")

LUSID Environment Initialised
LUSID version :  0.6.9549.0
Scope: As_at_DEMO-3ada-8fd6-27b6-4e
Code: PORT_0001


# 1. Define Helper Functions

Define various helper functions to help us create adjustments, to adjuset and set holdings, and to get holdings as of an effective date

In [43]:
def holding_adjustment(instrument_identifier: dict, units: int, price: int, currency : str) -> list:
     return [
        models.AdjustHoldingRequest(
            instrument_identifiers = instrument_identifier,
            tax_lots = [
                models.TargetTaxLotRequest(
                    units = units,
                    cost = models.CurrencyAndAmount(
                        amount = units,
                        currency = currency),
                    portfolio_cost = units * price,
                    price=price)
                    ]
        )
    ]

def adjust_holding(scope: str, code: str, holding_adjustment: list) -> object:
    return api_factory.build(lusid.api.TransactionPortfoliosApi).adjust_holdings(
        scope=scope,
        code=code,
        effective_at=datetime.datetime(2022, 3, 3, tzinfo=datetime.timezone.utc),
        adjust_holding_request=holding_adjustment)

def set_holding(scope: str, code: str, holding_adjustment: list) -> object:
    return api_factory.build(lusid.api.TransactionPortfoliosApi).set_holdings(
        scope=scope,
        code=code,
        effective_at=datetime.datetime(2022, 3, 3, tzinfo=datetime.timezone.utc),
        adjust_holding_request=holding_adjustment)


def get_historical_holdings_view(scope, code, from_date=None):
    from_date = from_date or datetime.datetime.now().date()

    historical_adjustments = []
    as_at = datetime.datetime.now(datetime.timezone.utc)
    
    get_next_adjustment = True
    values = []

    while get_next_adjustment:
        response = api_factory.build(lusid.api.TransactionPortfoliosApi).list_holdings_adjustments(
                    scope=scope,
                    code=code,
                    as_at=as_at

                )
        if response.values:
            if response.values[-1].version.as_at_date.date() >= from_date:
                as_at = response.values[-1].version.as_at_date - datetime.timedelta(microseconds=1)
                values.append(response.values[-1])
                continue
        get_next_adjustment = False

    #Grab all of our effective dates in our adjustments
    as_at_dates = [value.version.as_at_date for value in values]
    effective_at_dates = [value.effective_at for value in values]    

    for version, (as_at_date, effective_at_dates) in enumerate(zip(as_at_dates, effective_at_dates)):
        if as_at_date.date() >= from_date:
            response = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
                        scope=scope,
                        code=code,
                        as_at=as_at_date,
                        property_keys=["Instrument/default/Name"]
                    )
            historical_adjustments.append((response, effective_at_dates, len(as_at_dates) - version))
    return historical_adjustments

def display_holdings_summary(response, effective_date, version):
    # inspect holdings response for today
    hld = [i for i in response.values]
    
    names=[]
    amount=[]
    units=[]
    price=[]
    holding_type=[]
    
    for item in hld:
        
        names.append(item.properties['Instrument/default/Name'].value.label_value)
        amount.append(item.cost.amount)
        units.append(item.units)
        holding_type.append(item.holding_type)
        
    data={
        "version" : version,
        "names" : names,
        "amount" : amount,
        "units" : units,
        "holding_type" : holding_type,
        "effective_date" : effective_date,
        "as_at_date" : response.version.as_at_date
    }
    
    summary = pd.DataFrame(data=data)
    display(summary)




# 2. Insert Instruments into Instrument Master
For this example we are just using a basic GBP cash instrument, so ensure that we master that in LUSID's Instrument Master

In [3]:
# Call LUSID to upsert instruments
response = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=pd.DataFrame({"id":['GBP'], 'client_id':['GBP']}),
        mapping_required={"name": "id"},
        file_type="instruments",
        mapping_optional={},
        identifier_mapping={"ClientInternal": "client_id"}
)

# format response object
success, failed, errors = format_instruments_response(response)

pd.DataFrame(data=[{"success": len(success), "failed": len(failed), "errors": len(errors)}])

success  failed  errors
0        1       0       0

# 3. Create our test portfolio
Holdings are stored at a portfolio leveling, so create our test portfolio that will hold our holdings.

In [4]:
# call LUSID to create a portfolio or update details of an existing portfolio
mapping = {
    "portfolios": {
        "required": {
            "code": "Portfolio",
            "display_name":  "$As-at_POC",
            "base_currency": "$GBP",
            "created": "$2018-01-01T00:00:00+00:00"
        },
    }
}

response = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=pd.DataFrame({'Portfolio':['PORT_0001']}),
        mapping_required=mapping["portfolios"]["required"],
        file_type="portfolios",
        mapping_optional={},
)
# format response object
success, errors = format_portfolios_response(response)

pd.DataFrame(data=[{"success": len(success), "failed": len(failed), "errors": len(errors)}])

success  failed  errors
0        1       0       0

# 4. Insert our initial holding
The first step is to create our initial holding of 999 GBP Cash in our test portfolio, this uses set_holding instead of adjust holding as to start we have no holdings for this instrument in this portfolio

In [5]:
adjustment = holding_adjustment({'Instrument/default/Currency': 'GBP'}, 999, 1, 'GBP')

initial_holding = set_holding(scope, code, adjustment)

# 5. Update our holding
The next step is to keep building our timeline by adjusting our holding to 1100 1 hour later, notice the timestamps here that we are passing to adjust_holding, this is our effective datetime and our adjusted holding becomes effective from this time.

In [6]:
adjustment = holding_adjustment({'Instrument/default/Currency': 'GBP'}, 1100, 1, 'GBP')

holding_amendment_1 = adjust_holding(scope, code, adjustment)

# 6. Update our holding 
As above, keep building our timeline.  This time adjusting our holding to 1300

In [7]:
adjustment = holding_adjustment({'Instrument/default/Currency': 'GBP'}, 1300, 1, 'GBP')

holding_amendment_2 = adjust_holding(scope, code, adjustment)

# 7. View historical holding amounts since SOD
Use the list holding adjustment + get holding APIs in order to fetch our holdings as of an effective date to create a historical view of all of our holding from 2022, 3, 3 to present

In [44]:
historical_holdings = get_historical_holdings_view(scope, code, from_date=datetime.date(2022, 3, 3))

for item in historical_holdings:
    display_holdings_summary(*item)

version names  amount   units holding_type            effective_date  \
0        3   GBP  1300.0  1300.0            B 2022-03-03 00:00:00+00:00   

                        as_at_date  
0 2022-06-30 10:46:39.696474+00:00

version names  amount   units holding_type            effective_date  \
0        2   GBP  1100.0  1100.0            B 2022-03-03 00:00:00+00:00   

                        as_at_date  
0 2022-06-30 10:46:39.348166+00:00

version names  amount  units holding_type            effective_date  \
0        1   GBP   999.0  999.0            B 2022-03-03 00:00:00+00:00   

                        as_at_date  
0 2022-06-30 10:46:38.970223+00:00

# 8. Clean up 
Ensure that we tidy up after the example by deleting the portfolio

In [ ]:
_ = api_factory.build(lusid.api.PortfoliosApi).delete_portfolio(scope, code)